# A Transportation Problem Example -- Han Solo's Smuggling

Han Solo has three big crime lords he’s trying to keep happy and is planning three new storage bases from which to ship his smuggled goods. Each crime lord requires specific numbers of deliveries each day and each site can supply different numbers of deliveries. It costs Han 4 credits/parsec to travel between the storage sites and crime lords. Distances to each crime lord from each storage site are given below:

| Route  | Jabba | Pyke    | Sollima | Capacity | 
|-------:|------:|--------:|--------:|---------:|
|      1 | 50    |   25    |  84     | 24       | 
|      2 | 12    |   75    |  18     | 12       |
|      3 | 64    |   39    |  70     | 9        | 
|Cred Req| 16    |   20    |  9      | = 45     |

How should Han route deliveries to the crime lords while minimize his costs and meeting all demands?

In [1]:
using JuMP, Clp, NamedArrays

# create index sets of storage sites and crime lords
sites = [ 1,  2,  3]
crimelords = [:J, :P, :S]

# cost to ship from storage site to crime lord
cost_per_haul = 4
# NOTE: either 4 or 4*2 would be correct (with or without return trip -- just scales objective)

# create a NamedArray with indices of Sites (rows) and Crime Lords (columns). Elements are distances.
dist = NamedArray( [50 25 84; 12 75 18;64 39 70], (sites,crimelords), ("Sites","Crimelords") )
# each site has a max # of deliveries
supply = Dict(zip( sites, [24 12 9] ))
# each crime lord has a required number of deliveries
demand = Dict(zip( crimelords, [16 20 9] ))

m = Model(Clp.Optimizer)

@variable(m, x[sites,crimelords] >= 0) # x[i,j] is number of deliveries sent from site i to crimelord j

@constraint(m, sup[i in sites], sum(x[i,j] for j in crimelords) == supply[i] )   # supply constraint
@constraint(m, dem[j in crimelords], sum(x[i,j] for i in sites) == demand[j] )   # demand constraint

# minimize transportation cost
@objective(m, Min, cost_per_haul*sum( x[i,j]*dist[i,j] for i in sites, j in crimelords ) ) 

# solve this instance of the trasportation problem
optimize!(m)

# print out a nicely formatted solution
solution = NamedArray( 
    Int[value(x[i,j]) for i in sites, j in crimelords], 
    (sites,crimelords), 
    ("Sites","Crimelords") 
)
println( solution )
println()
println("Total cost will be \$", objective_value(m))


3×3 Named Matrix{Int64}
Sites ╲ Crimelords │ :J  :P  :S
───────────────────┼───────────
1                  │  4  20   0
2                  │  3   0   9
3                  │  9   0   0

Total cost will be $5896.0
Coin0506I Presolve 6 (0) rows, 9 (0) columns and 18 (0) elements
Clp0006I 0  Obj 0 Primal inf 89.999999 (6)
Clp0006I 6  Obj 5896
Clp0000I Optimal - objective value 5896
Clp0032I Optimal objective 5896 - 6 iterations time 0.012
